Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [4]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [5]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  print('old dataset shape:', dataset.shape)
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  print('new dataset shape:', dataset.shape)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  print('old labels shape:', labels.shape)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  print('new labels shape:', labels.shape)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

old dataset shape: (200000, 28, 28)
new dataset shape: (200000, 784)
old labels shape: (200000,)
new labels shape: (200000, 10)
old dataset shape: (10000, 28, 28)
new dataset shape: (10000, 784)
old labels shape: (10000,)
new labels shape: (10000, 10)
old dataset shape: (10000, 28, 28)
new dataset shape: (10000, 784)
old labels shape: (10000,)
new labels shape: (10000, 10)
Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [57]:
train_dataset[:1000, :].shape

(1000, 784)

In [8]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

In [58]:
graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random valued following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [59]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

def run_gradient_computation():
  with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
      # Run the computations. We tell .run() that we want to run the optimizer,
      # and get the loss value and the training predictions returned as numpy
      # arrays.
      _, l, predictions = session.run([optimizer, loss, train_prediction])
      if (step % 100 == 0):
        print('Loss at step %d: %f' % (step, l))
        print('Training accuracy: %.1f%%' % accuracy(
          predictions, train_labels[:train_subset, :]))
        # Calling .eval() on valid_prediction is basically like calling run(), but
        # just to get that one numpy array. Note that it recomputes all its graph
        # dependencies.
        print('Validation accuracy: %.1f%%' % accuracy(
          valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
    
%time run_gradient_computation()

Initialized
Loss at step 0: 16.547537
Training accuracy: 11.3%
Validation accuracy: 11.8%
Loss at step 100: 2.247135
Training accuracy: 72.2%
Validation accuracy: 71.1%
Loss at step 200: 1.848740
Training accuracy: 75.0%
Validation accuracy: 73.2%
Loss at step 300: 1.625857
Training accuracy: 76.0%
Validation accuracy: 73.8%
Loss at step 400: 1.469987
Training accuracy: 76.8%
Validation accuracy: 74.3%
Loss at step 500: 1.350899
Training accuracy: 77.4%
Validation accuracy: 74.4%
Loss at step 600: 1.255717
Training accuracy: 77.8%
Validation accuracy: 74.6%
Loss at step 700: 1.177489
Training accuracy: 78.4%
Validation accuracy: 74.7%
Loss at step 800: 1.111838
Training accuracy: 79.0%
Validation accuracy: 74.8%
Test accuracy: 82.4%
CPU times: user 1min 53s, sys: 12.3 s, total: 2min 5s
Wall time: 22.1 s


Initial values from the original notebook:

<pre>
Initialized
Loss at step 0 : 17.2939
Training accuracy: 10.8%
Validation accuracy: 13.8%
Loss at step 100 : 2.26903
Training accuracy: 72.3%
Validation accuracy: 71.6%
Loss at step 200 : 1.84895
Training accuracy: 74.9%
Validation accuracy: 73.9%
Loss at step 300 : 1.60701
Training accuracy: 76.0%
Validation accuracy: 74.5%
Loss at step 400 : 1.43912
Training accuracy: 76.8%
Validation accuracy: 74.8%
Loss at step 500 : 1.31349
Training accuracy: 77.5%
Validation accuracy: 75.0%
Loss at step 600 : 1.21501
Training accuracy: 78.1%
Validation accuracy: 75.4%
Loss at step 700 : 1.13515
Training accuracy: 78.6%
Validation accuracy: 75.4%
Loss at step 800 : 1.0687
Training accuracy: 79.2%
Validation accuracy: 75.6%
Test accuracy: 82.9%
</pre>

Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [60]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [61]:
num_steps = 3001

def run_sgd_computation():
  with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
      # Pick an offset within the training data, which has been randomized.
      # Note: we could use better randomization across epochs.
      offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
      # Generate a minibatch.
      batch_data = train_dataset[offset:(offset + batch_size), :]
      batch_labels = train_labels[offset:(offset + batch_size), :]
      # Prepare a dictionary telling the session where to feed the minibatch.
      # The key of the dictionary is the placeholder node of the graph to be fed,
      # and the value is the numpy array to feed to it.
      feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
      _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
      if (step % 500 == 0):
        print("Minibatch loss at step %d: %f" % (step, l))
        print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
        print("Validation accuracy: %.1f%%" % accuracy(
          valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    
%time run_sgd_computation()

Initialized
Minibatch loss at step 0: 18.757153
Minibatch accuracy: 6.2%
Validation accuracy: 10.5%
Minibatch loss at step 500: 1.822854
Minibatch accuracy: 75.0%
Validation accuracy: 75.3%
Minibatch loss at step 1000: 1.698213
Minibatch accuracy: 73.4%
Validation accuracy: 76.9%
Minibatch loss at step 1500: 1.259102
Minibatch accuracy: 74.2%
Validation accuracy: 76.9%
Minibatch loss at step 2000: 0.858849
Minibatch accuracy: 76.6%
Validation accuracy: 77.6%
Minibatch loss at step 2500: 1.020917
Minibatch accuracy: 79.7%
Validation accuracy: 78.2%
Minibatch loss at step 3000: 1.189150
Minibatch accuracy: 77.3%
Validation accuracy: 78.3%
Test accuracy: 85.7%
CPU times: user 7.18 s, sys: 3.22 s, total: 10.4 s
Wall time: 3.43 s


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

Logistic regression example without SGD: 1-hidden layer neural network with rectified linear units

In [11]:
def create_relu_graph(nb_hidden):
    graph = tf.Graph()
    with graph.as_default():
        # Input data.
        # Load the training, validation and test data into constants that are
        # attached to the graph.
        tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
        tf_train_labels = tf.constant(train_labels[:train_subset])
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)
  
        # hidden layer
        weights_1 = tf.Variable(
            tf.truncated_normal([image_size * image_size, nb_hidden]))
        biases_1 = tf.Variable(tf.zeros([nb_hidden]))
        hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
        
        # output layer
        weights_2 = tf.Variable(
            tf.truncated_normal([nb_hidden, num_labels]))
        biases_2 = tf.Variable(tf.zeros([num_labels]))
        logits = tf.matmul(hidden, weights_2) + biases_2

        # loss function
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
        # Optimizer.
        # We are going to find the minimum of this loss using gradient descent.
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
        # Predictions for the training, validation, and test data.
        # These are not part of training, but merely here so that we can report
        # accuracy figures as we train.
        def compute_prediction(dataset):
            return tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(dataset, weights_1) + biases_1), weights_2) + biases_2)
        
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = compute_prediction(tf_valid_dataset)
        test_prediction = compute_prediction(tf_test_dataset)

        return (graph, optimizer, loss, { 'train': train_prediction, 'valid': valid_prediction, 'test': test_prediction})

Compute accuracy for 10, 100, 500 and 1000 hidden nodes

In [12]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
        / predictions.shape[0])

def run_relu_computation(nb_hidden):
    print('')
    graph, optimizer, loss, preds = create_relu_graph(nb_hidden)
    with tf.Session(graph=graph) as session:
        # This is a one-time operation which ensures the parameters get initialized as
        # we described in the graph: random weights for the matrix, zeros for the
        # biases. 
        tf.initialize_all_variables().run()
        print('Initialized for {0} hidden relu'.format(nb_hidden))
        for step in range(num_steps):
            # Run the computations. We tell .run() that we want to run the optimizer,
            # and get the loss value and the training predictions returned as numpy
            # arrays.
            _, l, predictions = session.run([optimizer, loss, preds['train']])
            if (step % 100 == 0):
                print('Loss at step %d: %f' % (step, l))
                print('Training accuracy: %.1f%%' % accuracy(
                predictions, train_labels[:train_subset, :]))
                # Calling .eval() on valid_prediction is basically like calling run(), but
                # just to get that one numpy array. Note that it recomputes all its graph
                # dependencies.
                print('Validation accuracy: %.1f%%' % accuracy(
                    preds['valid'].eval(), valid_labels))
        print('Test accuracy: %.1f%%' % accuracy(
            preds['test'].eval(), test_labels))
    
%time run_relu_computation(10)
%time run_relu_computation(100)
%time run_relu_computation(500)
%time run_relu_computation(1000)


Initialized for 10 hidden relu
Loss at step 0: 29.936613
Training accuracy: 11.6%
Validation accuracy: 23.2%
Loss at step 100: 1.469934
Training accuracy: 57.7%
Validation accuracy: 57.6%
Loss at step 200: 1.270297
Training accuracy: 63.4%
Validation accuracy: 62.3%
Loss at step 300: 1.252973
Training accuracy: 67.2%
Validation accuracy: 68.4%
Loss at step 400: 1.211750
Training accuracy: 64.4%
Validation accuracy: 59.4%
Loss at step 500: 0.898835
Training accuracy: 75.2%
Validation accuracy: 73.3%
Loss at step 600: 0.857774
Training accuracy: 75.8%
Validation accuracy: 74.0%
Loss at step 700: 0.824533
Training accuracy: 76.8%
Validation accuracy: 74.5%
Loss at step 800: 0.785283
Training accuracy: 77.4%
Validation accuracy: 75.2%
Test accuracy: 83.2%
CPU times: user 1min 53s, sys: 13.2 s, total: 2min 6s
Wall time: 24.5 s

Initialized for 100 hidden relu
Loss at step 0: 82.670334
Training accuracy: 14.9%
Validation accuracy: 30.8%
Loss at step 100: 2.942972
Training accuracy: 76.8%
Va

Logistic regression example with SGD: 1-hidden layer neural network with rectified linear units

In [13]:
batch_size = 128

def create_sgd_relu_graph(nb_hidden):
    graph = tf.Graph()
    with graph.as_default():
        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)
  
        # hidden layer
        weights_1 = tf.Variable(
            tf.truncated_normal([image_size * image_size, nb_hidden]))
        biases_1 = tf.Variable(tf.zeros([nb_hidden]))
        hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
        
        # output layer
        weights_2 = tf.Variable(
            tf.truncated_normal([nb_hidden, num_labels]))
        biases_2 = tf.Variable(tf.zeros([num_labels]))
        logits = tf.matmul(hidden, weights_2) + biases_2

        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
        # Optimizer.
        # We are going to find the minimum of this loss using gradient descent.
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
        # Predictions for the training, validation, and test data.
        # These are not part of training, but merely here so that we can report
        # accuracy figures as we train.
        def compute_prediction(dataset):
            return tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(dataset, weights_1) + biases_1), weights_2) + biases_2)
        
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = compute_prediction(tf_valid_dataset)
        test_prediction = compute_prediction(tf_test_dataset)
        
    return (graph, optimizer, loss,
            { 'dataset': tf_train_dataset, 'labels': tf_train_labels },
            { 'train': train_prediction, 'valid': valid_prediction, 'test': test_prediction })

Compute accuracy for 10, 100, 500, 1000, 5000 and 10000 hidden nodes

In [16]:
num_steps = 6001

def run_sgd_relu_computation(nb_hidden):
    print('')
    graph, optimizer, loss, feed, preds = create_sgd_relu_graph(nb_hidden)
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print('Initialized for {0} hidden relu'.format(nb_hidden))
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {feed['dataset'] : batch_data, feed['labels'] : batch_labels}
            _, l, predictions = session.run(
                [optimizer, loss, preds['train']], feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(
                    preds['valid'].eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(preds['test'].eval(), test_labels))
    
%time run_sgd_relu_computation(10)
%time run_sgd_relu_computation(100)
%time run_sgd_relu_computation(500)
%time run_sgd_relu_computation(1000)
%time run_sgd_relu_computation(5000)
%time run_sgd_relu_computation(10000)


Initialized for 10 hidden relu
Minibatch loss at step 0: 30.357452
Minibatch accuracy: 6.2%
Validation accuracy: 22.0%
Minibatch loss at step 500: 1.397497
Minibatch accuracy: 53.9%
Validation accuracy: 58.4%
Minibatch loss at step 1000: 0.933471
Minibatch accuracy: 74.2%
Validation accuracy: 73.0%
Minibatch loss at step 1500: 1.113285
Minibatch accuracy: 70.3%
Validation accuracy: 72.7%
Minibatch loss at step 2000: 0.666694
Minibatch accuracy: 79.7%
Validation accuracy: 77.4%
Minibatch loss at step 2500: 0.674479
Minibatch accuracy: 82.0%
Validation accuracy: 78.0%
Minibatch loss at step 3000: 0.896895
Minibatch accuracy: 75.8%
Validation accuracy: 78.8%
Test accuracy: 86.0%
CPU times: user 9.14 s, sys: 4.02 s, total: 13.2 s
Wall time: 4.57 s

Initialized for 100 hidden relu
Minibatch loss at step 0: 82.070587
Minibatch accuracy: 4.7%
Validation accuracy: 26.4%
Minibatch loss at step 500: 1.523089
Minibatch accuracy: 71.9%
Validation accuracy: 70.3%
Minibatch loss at step 1000: 2.681